<a href="https://colab.research.google.com/github/SonDaYeong/CP1-YamPick/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 한글 폰트 설치하기 (꼭! 설치가 완료되면 [런타임 다시 시작]을 누르고 다시 실행하기)
!apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정하기
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (18.4 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
### 이미지 전처리
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

PATH = '/content/drive/MyDrive/CS/CP1/'
image_size = 150

original_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    # https://keras.io/api/preprocessing/image/
    # rotation_range=30,  #회전 범위
    width_shift_range=0.1, #그림을 수평 수직 또는 랜덤하게 이동 -> 가로 세로에 대한 비율값
    height_shift_range=0.1,
    rescale=1./255, # 이미지 스케일링을 통해 0-1사이의 범위로 변환
    shear_range=0.2, # 층 밀림 변환
    zoom_range=0.2, #확대 축소 범위 
    horizontal_flip=True # True : 50% 확률로 이미지를 수평으로 뒤집음 
    # fill_mode='nearest' # 이미지 회전, 이동, 축소시 생기는 공간을 채우는 방식 
)

train_generator = train_datagen.flow_from_directory(
        PATH+'train',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')
        # class_mode='int')

## 이미지 증강 X
train_origin_generator = original_datagen.flow_from_directory(
        PATH+'train',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')

val_generator = original_datagen.flow_from_directory(
        PATH+'test',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')
        # class_mode='int')


Found 3192 images belonging to 4 classes.
Found 3192 images belonging to 4 classes.
Found 813 images belonging to 4 classes.


In [ ]:
class_names = train_generator.class_indices
class_names = {value:key for key, value in class_names.items()}
class_names

{0: '갈비찜', 1: '갈비탕', 2: '갈치구이', 3: '갈치조림'}

In [ ]:
num_classes = train_generator.num_classes
epoch = 5

In [ ]:
### vgg16 모델 활용
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
vgg.trainable=False

vgg_ft = VGG16(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
vgg_ft.trainable=False

In [ ]:
# vgg 기본
vgg_ml = Sequential([
    vgg,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# vgg 기본
vgg_GAP_ml = Sequential([
    vgg,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# drop_out 살리고 확인해보자.
vgg_dropout_5_ml = Sequential([
    vgg,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# drop_out 살리고 확인해보자.
vgg_dropout_2_ml = Sequential([
    vgg,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])


# # fine tuning 진행 
vgg_ft.trainable=True
for layer in vgg.layers[:15]:
    layer.trainable = False

vgg_ft_ml = Sequential([
    vgg_ft,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])


vgg_ft_dropout_ml = Sequential([
    vgg_ft,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
resnet_ft = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))

for layer in resnet.layers:
    layer.trainable = False

# vgg 기본
resnet_ml = Sequential([
    resnet,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# vgg 기본
resnet_GAP_ml = Sequential([
    resnet,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# drop_out 살리고 확인해보자.
resnet_dropout_5_ml = Sequential([
    resnet,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# drop_out 살리고 확인해보자.
resnet_dropout_2_ml = Sequential([
    resnet,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

# fine tuning 진행 
resnet_ft.trainable=True
for layer in resnet_ft.layers[:15]:
    layer.trainable = False

resnet_ft_ml = Sequential([
    resnet_ft,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

resnet_ft_dropout_ml = Sequential([
    resnet_ft,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [ ]:
##
def image_augmentation_data(model):
  model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
  # batch_size 128.. 적절한 batch size를 찾아야함
  model_hist = model.fit_generator( 
                    train_generator,
                    validation_data=val_generator,
                    # batch_size = batch_size,
                    epochs=epoch
                  )
  return model_hist

# 데이터 증강 x
def image_origin_data(model):
  model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])
  # batch_size 128.. 적절한 batch size를 찾아야함
  model_hist = model.fit_generator( 
                    train_origin_generator,
                    validation_data=val_generator,
                    # batch_size = batch_size,
                    epochs=epoch
                  )

  return model_hist

In [ ]:
vgg_ml_list = [vgg_ml, vgg_GAP_ml, vgg_dropout_5_ml, vgg_dropout_2_ml, vgg_ft_ml, vgg_ft_dropout_ml]
resnet_ml_list = [resnet_ml, resnet_GAP_ml, resnet_dropout_5_ml, resnet_dropout_2_ml, resnet_ft_ml, resnet_ft_dropout_ml]
vgg_info_list = ["기본 VGG Model & Flatten", "기본 VGG Model & Pooling", "VGG + Dropout(0.5)", "VGG + Dropout(0.2)", "VGG + Finetuning", "VGG + FineTuning + Dropout"]
resnet_info_list = ["기본 Resnet Model & Flatten", "기본 Resnet Model & Pooling", "Resnet + Dropout(0.5)", "Resnet + Dropout(0.2)", "Resnet + Finetuning", "Resnet + FineTuning + Dropout"]

In [ ]:
vgg_ml_ori_list = vgg_ml_list.deepcopy()

In [ ]:

vg_ag_data_hist, vg_ori_data_hist = [], []
for idx, model in enumerate(vgg_ml_list):
  print(vgg_info_list[idx])
  vg_ag_data_hist.append((model))
  print("=========================image_augmentation_data(model))
  vg_ag_data_hist.append(image_augmentation_data=========================")

resnet_ag_data_hist, resnet_ori_data_hist = [], []
for idx, model in enumerate(resnet_ml_list):
  print(resnet_info_list[idx])
  resnet_ag_data_hist.append(image_augmentation_data(model))
  resnet_ori_data_hist.append(image_origin_data(model))
  print("==================================================")


기본 VGG Model & Flatten


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
100/100 [==============================] - 54s 538ms/step - loss: 0.7548 - accuracy: 0.6958 - val_loss: 0.5319 - val_accuracy: 0.7995
Epoch 2/5
100/100 [==============================] - 55s 550ms/step - loss: 0.4895 - accuracy: 0.8130 - val_loss: 0.4815 - val_accuracy: 0.8229
Epoch 3/5
100/100 [==============================] - 53s 534ms/step - loss: 0.4352 - accuracy: 0.8268 - val_loss: 0.4804 - val_accuracy: 0.8130
Epoch 4/5
100/100 [==============================] - 55s 550ms/step - loss: 0.3818 - accuracy: 0.8477 - val_loss: 0.4715 - val_accuracy: 0.8327
Epoch 5/5
100/100 [==============================] - 53s 533ms/step - loss: 0.3378 - accuracy: 0.8741 - val_loss: 0.5163 - val_accuracy: 0.8143


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
100/100 [==============================] - 41s 402ms/step - loss: 0.2738 - accuracy: 0.8957 - val_loss: 0.5689 - val_accuracy: 0.8057
Epoch 2/5
100/100 [==============================] - 39s 388ms/step - loss: 0.1571 - accuracy: 0.9502 - val_loss: 0.4991 - val_accuracy: 0.8499
Epoch 3/5
100/100 [==============================] - 39s 388ms/step - loss: 0.1048 - accuracy: 0.9718 - val_loss: 0.5143 - val_accuracy: 0.8401
Epoch 4/5
100/100 [==============================] - 40s 399ms/step - loss: 0.0581 - accuracy: 0.9893 - val_loss: 0.5149 - val_accuracy: 0.8585
Epoch 5/5
100/100 [==============================] - 38s 385ms/step - loss: 0.0423 - accuracy: 0.9934 - val_loss: 0.5483 - val_accuracy: 0.8462
기본 VGG Model & Pooling
Epoch 1/5
100/100 [==============================] - 55s 549ms/step - loss: 0.9438 - accuracy: 0.6275 - val_loss: 0.7176 - val_accuracy: 0.7331
Epoch 2/5
100/100 [==============================] - 56s 556ms/step - loss: 0.6732 - accuracy: 0.7397 - val_loss:

In [ ]:
import copy
vg_ag_data_hist_fin, vg_ori_data_hist_fin = [], []
for idx, model in enumerate(vgg_ml_list):
  print(vgg_info_list[idx])
  print("==============augmentation==============")
  vg_ag_data_hist_fin.append(image_augmentation_data(copy.deepcopy(model)))
  print("===============origin====================")
  vg_ori_data_hist_fin.append(image_origin_data(copy.deepcopy(model)))
  print("==================================================")

resnet_ag_data_hist_fin, resnet_ori_data_hist_fin = [], []
for idx, model in enumerate(resnet_ml_list):
  print(resnet_info_list[idx])
  print("==============augmentation==============")
  resnet_ag_data_hist_fin.append(image_augmentation_data(copy.deepcopy(model)))
  print("===============origin====================")
  resnet_ori_data_hist_fin.append(image_origin_data(copy.deepcopy(model)))
  print("==================================================")


기본 VGG Model & Flatten
==============augmentation==============


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
100/100 [==============================] - 55s 540ms/step - loss: 0.3094 - accuracy: 0.8853 - val_loss: 0.5138 - val_accuracy: 0.8401
Epoch 2/5
100/100 [==============================] - 54s 537ms/step - loss: 0.2697 - accuracy: 0.8997 - val_loss: 0.5429 - val_accuracy: 0.8278
Epoch 3/5
100/100 [==============================] - 54s 544ms/step - loss: 0.2533 - accuracy: 0.8982 - val_loss: 0.4917 - val_accuracy: 0.8364
Epoch 4/5
100/100 [==============================] - 53s 528ms/step - loss: 0.2645 - accuracy: 0.8907 - val_loss: 0.5973 - val_accuracy: 0.8069
Epoch 5/5
100/100 [==============================] - 54s 538ms/step - loss: 0.2296 - accuracy: 0.9142 - val_loss: 0.4952 - val_accuracy: 0.8364
===============origin====================


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
100/100 [==============================] - 54s 530ms/step - loss: 0.3194 - accuracy: 0.8850 - val_loss: 0.5653 - val_accuracy: 0.8266
Epoch 2/5
100/100 [==============================] - 54s 539ms/step - loss: 0.2764 - accuracy: 0.8963 - val_loss: 0.6301 - val_accuracy: 0.8044
Epoch 3/5
100/100 [==============================] - 53s 527ms/step - loss: 0.2572 - accuracy: 0.8997 - val_loss: 0.5401 - val_accuracy: 0.8303
Epoch 4/5
100/100 [==============================] - 54s 543ms/step - loss: 0.2337 - accuracy: 0.9085 - val_loss: 0.4762 - val_accuracy: 0.8376
Epoch 5/5
100/100 [==============================] - 54s 537ms/step - loss: 0.2541 - accuracy: 0.9013 - val_loss: 0.4789 - val_accuracy: 0.8352
기본 VGG Model & Pooling
==============augmentation==============


...

Epoch 1/5
100/100 [==============================] - 50s 446ms/step - loss: 0.3725 - accuracy: 0.8678 - val_loss: 0.6009 - val_accuracy: 0.8327
Epoch 2/5
100/100 [==============================] - 42s 417ms/step - loss: 0.3075 - accuracy: 0.9076 - val_loss: 8.3988 - val_accuracy: 0.5547
Epoch 3/5
100/100 [==============================] - 42s 417ms/step - loss: 0.2267 - accuracy: 0.9289 - val_loss: 0.5209 - val_accuracy: 0.8721
Epoch 4/5
100/100 [==============================] - 41s 413ms/step - loss: 0.1397 - accuracy: 0.9511 - val_loss: 0.6094 - val_accuracy: 0.8684
Epoch 5/5
100/100 [==============================] - 42s 417ms/step - loss: 0.0835 - accuracy: 0.9731 - val_loss: 0.5436 - val_accuracy: 0.8659
Resnet + FineTuning + Dropout
==============augmentation==============


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/5
100/100 [==============================] - 64s 591ms/step - loss: 0.6464 - accuracy: 0.7932 - val_loss: 1.9052 - val_accuracy: 0.6605
Epoch 2/5
100/100 [==============================] - 56s 559ms/step - loss: 0.4924 - accuracy: 0.8437 - val_loss: 0.8239 - val_accuracy: 0.7355
Epoch 3/5
100/100 [==============================] - 56s 556ms/step - loss: 0.4505 - accuracy: 0.8490 - val_loss: 0.4043 - val_accuracy: 0.8659
Epoch 4/5
100/100 [==============================] - 56s 556ms/step - loss: 0.3674 - accuracy: 0.8759 - val_loss: 0.4607 - val_accuracy: 0.8229
Epoch 5/5
100/100 [==============================] - 56s 555ms/step - loss: 0.3388 - accuracy: 0.8853 - val_loss: 0.3790 - val_accuracy: 0.8684
===============origin====================


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
100/100 [==============================] - 54s 492ms/step - loss: 0.4136 - accuracy: 0.8609 - val_loss: 0.6611 - val_accuracy: 0.8598
Epoch 2/5
100/100 [==============================] - 43s 429ms/step - loss: 0.2724 - accuracy: 0.9145 - val_loss: 0.6121 - val_accuracy: 0.8438
Epoch 3/5
100/100 [==============================] - 42s 419ms/step - loss: 0.1985 - accuracy: 0.9330 - val_loss: 0.6802 - val_accuracy: 0.8290
Epoch 4/5
100/100 [==============================] - 43s 427ms/step - loss: 0.1616 - accuracy: 0.9514 - val_loss: 0.8783 - val_accuracy: 0.8069
Epoch 5/5
100/100 [==============================] - 42s 421ms/step - loss: 0.4417 - accuracy: 0.8678 - val_loss: 48.0874 - val_accuracy: 0.3702


In [ ]:
### model overfitting, underfitting 확인

import matplotlib.pyplot as plt

def model_accuracy(model, model_info):
    
    print("============",model_info,"============")
    
    plt.plot(model.history['accuracy'])
    # plt.plot(model.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    plt.plot(model.history['loss'])
    # plt.plot(model.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()



In [ ]:

for idx, hist in enumerate(vg_ag_data_hist):
  model_accuracy(hist, "Augmentation + " + vgg_info_list[idx])
  model_accuracy(vg_ori_data_hist, "Augmentation + " + vgg_info_list[idx])
  
intel gpu

NameError: ignored

In [ ]:

for idx, hist in enumerate(vg_ag_data_hist_fin):
  model_accuracy(hist, "Augmentation + " + vgg_info_list[idx])
  model_accuracy(vg_ori_data_hist_fin, "Augmentation + " + vgg_info_list[idx])
  


In [ ]:

for idx, hist in enumerate(resnet_ag_data_hist):
  model_accuracy(hist, "Augmentation + " + resnet_info_list[idx])
  model_accuracy(resnet_ori_data_hist, "Augmentation + " + resnet_info_list[idx])
  


In [ ]:

for idx, hist in enumerate(resnet_ag_data_hist_fin):
  model_accuracy(hist, "Augmentation + " + resnet_info_list[idx])
  model_accuracy(resnet_ori_data_hist_fin, "Augmentation + " + resnet_info_list[idx])
  
